# Abstract

Find out how neighborhood can affect rental pricing

# Load Data

Data files are stored on google drive.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
import numpy as np

file_path = '/content/drive/My Drive/MAR 653/Final Project/data/{}'

# Listing data
listings_summary_df = pd.read_csv(file_path.format('listings.csv'))
listings_detail_df = pd.read_csv(file_path.format('listings.csv.gz'), compression='gzip')

# Prizm data
prizm_zipcode_df = pd.read_excel(file_path.format('prizm_attributes.xlsx'), sheet_name='zipcodes')
prizm_attributes_df = pd.read_excel(file_path.format('prizm_attributes.xlsx'), sheet_name='attributes')
prizm_zipcode_df.replace('', np.nan, inplace=True)
prizm_zipcode_df.dropna(axis=0, inplace=True)


# Data Transformation

The output dataframe, **listings_neighbourhood_df**, shall contain the following columns: **price**, **price_normalized**, **bedrooms**, **beds**, **upper**, **upper_middle**, **lower_middle**, and **lower**.

In [0]:
# Get price from listings_summary_df
# Get zipcode, bedrooms, and beds from listing_detail_df
import numpy as np

listings_neighbourhood_df = listings_summary_df[['price']]
listings_neighbourhood_df['zipcode'] = listings_detail_df['zipcode']
listings_neighbourhood_df['bedrooms'] = listings_detail_df['bedrooms']
listings_neighbourhood_df['beds'] = listings_detail_df['beds']

listings_neighbourhood_df = listings_neighbourhood_df.drop(9446)
listings_neighbourhood_df = listings_neighbourhood_df.drop(55)
listings_neighbourhood_df = listings_neighbourhood_df.drop(1660)
listings_neighbourhood_df = listings_neighbourhood_df.drop(4279)
listings_neighbourhood_df = listings_neighbourhood_df.drop(12377)
listings_neighbourhood_df = listings_neighbourhood_df.drop(15110)
listings_neighbourhood_df = listings_neighbourhood_df.drop(20543)
listings_neighbourhood_df = listings_neighbourhood_df.drop(21968)
listings_neighbourhood_df = listings_neighbourhood_df.drop(24371)
listings_neighbourhood_df = listings_neighbourhood_df.drop(25125)
listings_neighbourhood_df = listings_neighbourhood_df.drop(28562)
listings_neighbourhood_df = listings_neighbourhood_df.drop(31986)
listings_neighbourhood_df = listings_neighbourhood_df.drop(33587)
listings_neighbourhood_df = listings_neighbourhood_df.drop(39722)
listings_neighbourhood_df = listings_neighbourhood_df.drop(40122)
listings_neighbourhood_df = listings_neighbourhood_df.drop(42478)
listings_neighbourhood_df = listings_neighbourhood_df.drop(50491)
listings_neighbourhood_df.replace('', np.nan, inplace=True)
listings_neighbourhood_df.dropna(axis=0, inplace=True)


def clean_zipcode(x):
  if '\n' in x:
    return x.split('\n')[0]
  elif 'NY' in x:
    return x.split(' ')[1]
  elif '-' in x:
    return x.split('-')[0]
  else:
    return x
listings_neighbourhood_df['zipcode'] = listings_neighbourhood_df['zipcode'].apply(clean_zipcode)

In [0]:
# Normalize Price
def set_price_level(x):
    if 69 >= x:
      return 'low'
    elif 105 >= x:
      return 'low medium' 
    elif 175 >= x:
      return 'high medium'
    else:
      return 'high'

listings_neighbourhood_df['price_normalized'] = listings_neighbourhood_df['price'].apply(set_price_level)

In [0]:
# Calculate the averaged prizm score by zipcode
prizm_zipcode_df['average_score'] = -1
for i, row in prizm_zipcode_df.iterrows():
  attribute_id_arr = row['attribute_id'].split(',')
  sum = 0
  count = len(attribute_id_arr)
  for id in attribute_id_arr:
    match_row = prizm_attributes_df.loc[prizm_attributes_df['id']==int(id)]
    row_i = match_row.index[0]
    sum += match_row['score'][row_i]
  prizm_zipcode_df.loc[i, 'average_score'] = sum / count

In [0]:
# Calculate upper, upper_middle, lower_middle, lower
listings_neighbourhood_df['upper'] = 0
listings_neighbourhood_df['upper_middle'] = 0
listings_neighbourhood_df['lower_middle'] = 0
listings_neighbourhood_df['lower'] = 0

for i, row in listings_neighbourhood_df.iterrows():
  match_row = prizm_zipcode_df.loc[prizm_zipcode_df['zipcode']==int(row['zipcode'])]
  average_score = match_row['average_score'][match_row.index[0]]
  if average_score >= 4:
    listings_neighbourhood_df.loc[i, 'upper'] = 1
  elif average_score >=3:
    listings_neighbourhood_df.loc[i, 'upper_middle'] = 1
  elif average_score >= 2:
    listings_neighbourhood_df.loc[i, 'lower_middle'] = 1
  else:
    listings_neighbourhood_df.loc[i, 'lower'] = 1


In [0]:
# drop zipcode
listings_neighbourhood_df.drop(columns=['zipcode'], inplace=True)

In [9]:
listings_neighbourhood_df.head()

,price,bedrooms,beds,price_normalized,upper,upper_middle,lower_middle,lower
0,225,0.0,1.0,high,0,1,0,0
1,89,1.0,4.0,low medium,0,1,0,0
2,200,1.0,1.0,high,0,1,0,0
3,60,1.0,1.0,low,0,1,0,0
4,79,1.0,1.0,low medium,0,1,0,0


# Cross Tabulation

Create a cross tabulation between price_normalized and neighborhood (i.e. upper, upper_middle, lower_middle, lower). The value is the sum of rentals in each neighborhood.

In [16]:
# Create cross tabulation dataframe
import pandas as pd

data_arr = []

# high
listing_sum = listings_neighbourhood_df[listings_neighbourhood_df['price_normalized'] == 'high'][['upper','upper_middle','lower_middle','lower']].sum()
data_arr.append([listing_sum['upper'], listing_sum['upper_middle'], listing_sum['lower_middle'], listing_sum['lower']])

# high medium
listing_sum = listings_neighbourhood_df[listings_neighbourhood_df['price_normalized'] == 'high medium'][['upper','upper_middle','lower_middle','lower']].sum()
data_arr.append([listing_sum['upper'], listing_sum['upper_middle'], listing_sum['lower_middle'], listing_sum['lower']])

# low medium
listing_sum = listings_neighbourhood_df[listings_neighbourhood_df['price_normalized'] == 'low medium'][['upper','upper_middle','lower_middle','lower']].sum()
data_arr.append([listing_sum['upper'], listing_sum['upper_middle'], listing_sum['lower_middle'], listing_sum['lower']])

# low
listing_sum = listings_neighbourhood_df[listings_neighbourhood_df['price_normalized'] == 'low'][['upper','upper_middle','lower_middle','lower']].sum()
data_arr.append([listing_sum['upper'], listing_sum['upper_middle'], listing_sum['lower_middle'], listing_sum['lower']])

cross_tabulation_df = pd.DataFrame(data=data_arr, index=['high', 'high_medium', 'low_medium', 'low'], columns=['upper', 'upper_middle', 'lower_middle', 'lower'])
cross_tabulation_df.head()

,upper,upper_middle,lower_middle,lower
high,2958,8947,436,0
high_medium,2215,9413,930,0
low_medium,1252,9641,1417,0
low,762,9780,2186,0
